# US Name
Model estimate Estimate internal finance and pollution emission firm level

# Description
None
# Metadata
- Key: 488_Financial_dependency_pollution
- Epic: Models
- US: Evaluate econometrics model
- Task tag: #internal-finance, #training-Financial-dependency-pollution
- Analytics reports: 
# Input
## Table/file
**Name**
- asif_financial_ratio_baseline_firm
- china_firm_pollution_data
**Github**
- https://github.com/thomaspernet/Financial_dependency_pollution/blob/master/02_data_analysis/01_model_train_evaluate/00_estimate_fin_ratio/08_firm_level_estimation_pollution.md



# Connexion server

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_s3 import service_s3
from awsPy.aws_glue import service_glue
from pathlib import Path
import pandas as pd
import numpy as np
#import seaborn as sns
import os, shutil, json
import sys

from sklearn import preprocessing

le = preprocessing.LabelEncoder()

path = os.getcwd()
parent_path = str(Path(path).parent.parent.parent)


name_credential = 'financial_dep_SO2_accessKeys.csv'
region = 'eu-west-2'
bucket = 'datalake-london'
path_cred = "{0}/creds/{1}".format(parent_path, name_credential)

In [2]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = False)
glue = service_glue.connect_glue(client = client) 

In [3]:
pandas_setting = True
if pandas_setting:
    #cm = sns.light_palette("green", as_cmap=True)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

In [4]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'


# Load tables

Since we load the data as a Pandas DataFrame, we want to pass the `dtypes`. We load the schema from Glue to guess the types

- 1=state -> 110 141 143 151
- 2=collective -> 120 130 142 149
- 3=private -171 172 173 174 190
- 4=foreign- 210 220 230 240
- 5=Hong Kong, Macau and Taiwan (4 and 5 can be combined into a single "foreign" category - 310 320 330 340

In [5]:
db = 'environment'
table = 'china_financial_dependency_pollution'

In [6]:
dtypes = {}
schema = (glue.get_table_information(database = db,
                           table = table)
          ['Table']['StorageDescriptor']['Columns']
         )
for key, value in enumerate(schema):
    if value['Type'] in ['varchar(12)',
                         'varchar(3)',
                        'varchar(14)', 'varchar(11)']:
        format_ = 'string'
    elif value['Type'] in ['decimal(21,5)', 'double', 'bigint', 'int', 'float']:
        format_ = 'float'
    else:
        format_ = value['Type'] 
    dtypes.update(
        {value['Name']:format_}
    )

In [7]:
download_data = True
filename = 'df_{}'.format(table)
full_path_filename = 'SQL_OUTPUT_ATHENA/CSV/{}.csv'.format(filename)
path_local = os.path.join(str(Path(path).parent.parent.parent), 
                              "00_data_catalog/temporary_local_data")
df_path = 'df_asif.csv'#os.path.join(path_local, filename + '.csv')
if download_data:
    
    s3 = service_s3.connect_S3(client = client,
                          bucket = bucket, verbose = False)
    query = """
    SELECT *,CASE WHEN rd_tot_asset IS NULL THEN -1000 
    WHEN rd_tot_asset < 0 THEN 0
    ELSE rd_tot_asset END AS rd_tot_asset_trick,
     CASE 
WHEN ownership_new in (1) THEN 'SOE' 
ELSE 'NOT_SOE' END AS SOE,
 CASE 
WHEN ownership_new in (4,5) THEN 'FOREIGN' 
ELSE 'NOT_FOREIGN' END AS FOREIGN,
concat(indu_2,'-',year) as fe_indu2_year,
concat(china_financial_dependency_pollution.geocode4_corr,'-',year) as fe_city_year
from environment.china_financial_dependency_pollution
LEFT JOIN policy.china_city_tcz_spz
        ON china_financial_dependency_pollution.geocode4_corr = china_city_tcz_spz.geocode4_corr
WHERE asset_tangibility_tot_asset IS NOT NULL and
    sales IS NOT NULL and
    total_asset IS NOT NULL and 
    cashflow_to_tangible >0
ORDER BY firm, year
    """.format(db, table)
    df = (s3.run_query(
            query=query,
            database=db,
            s3_output='SQL_OUTPUT_ATHENA',
            filename=filename,  # Add filename to print dataframe
            destination_key='SQL_OUTPUT_ATHENA/CSV',  #Use it temporarily
            dtype = dtypes
        )
          .assign(
              tcz = lambda x: x['tcz'].fillna(0).astype('int').astype('str'),
              spz = lambda x: x['spz'].fillna(0).astype('int').astype('str'),
              fe_fo=lambda x: le.fit_transform(x["firm"].astype('str')),
              fe_indu2_year=lambda x: le.fit_transform(x["fe_indu2_year"].astype('str')),
              fe_city_year=lambda x: le.fit_transform(x["fe_city_year"].astype('str')),
            )
                )
    #s3.download_file(
    #    key = full_path_filename
    #)
    #shutil.move(
    #    filename + '.csv',
    #    os.path.join(path_local, filename + '.csv')
    #
    #s3.remove_file(full_path_filename)
    #df.head()
    #df.to_csv(df_path)

In [8]:
query= """
SELECT * FROM "industry"."china_credit_constraint"
"""
df_credit = (s3.run_query(
            query=query,
            database=db,
            s3_output='SQL_OUTPUT_ATHENA',
            filename=filename,  # Add filename to print dataframe
            destination_key='SQL_OUTPUT_ATHENA/CSV',  #Use it temporarily
            dtype = dtypes
        )
     )
df_credit.dtypes

cic                      int64
industry_name           object
financial_dep_china    float64
financial_dep_us       float64
hum_cap_us             float64
phys_cap_us            float64
liquidity_need_us      float64
rd_intensity_us        float64
foreign_share_us       float64
dtype: object

In [9]:
df_credit.head(1)

,cic,industry_name,financial_dep_china,financial_dep_us,hum_cap_us,phys_cap_us,liquidity_need_us,rd_intensity_us,foreign_share_us
0,16,Tobacco,-1.54,-0.45,1.35,0.02,0.25,0.0,0.02


In [10]:
df.shape

(104699, 93)

In [11]:
df = (
    df
    .merge(df_credit.rename(columns = {'cic':'indu_2'}), how= 'inner')
    .assign(
        constraint = lambda x: x['financial_dep_china'] > -0.44,
        constraint_1 = lambda x: x['financial_dep_china'] > -0.26
    )
)

In [12]:
df.shape

(104285, 103)

In [13]:
df[['quick_ratio']].isna().sum()

quick_ratio    0
dtype: int64

In [14]:
df.head(1)

,year,firm,name,geocode4_corr,province_en,cic_adj,cic03,indu_2,ownership_new,age,tfp_op,tfp_lp,output,outputdefl,sales,employment,capital,current_asset,tofixed,error,total_liabilities,total_asset,total_right,intangible,tangible,net_non_current,cashflow,current_ratio,quick_ratio,liabilities_tot_asset,sales_tot_asset,investment_tot_asset,rd_tot_asset,asset_tangibility_tot_asset,cashflow_tot_asset,cashflow_to_tangible,return_to_sale,coverage_ratio,liquidity,total_industrialwater_used,total_freshwater_used,gyqs,total_repeatedwater_used,total_coal_used,rlmxf,ylmxf,rlmpjlf,rlyxf,zyxf,cyxf,rlypjlf,zypjlf,clean_gas_used,waste_water,cod,ad,waste_gas,so2,nox,smoke_dust,soot,yfc,gyfscll,hxxyqcl,xzssqcl,adqcl,eyhlqcl,dyhwqcl,ycqcl,gyfcqcl,dwastewater_equip,fszlssnl,fszlssfee,dwastegas_equip,dso2_equip,fqzlssnl,tlssnl,hxxycsl,adcsl,eyhlcsl,dyhwcsl,yfccsl,province,city,geocode4_corr.1,tcz,spz,rd_tot_asset_trick,SOE,FOREIGN,fe_indu2_year,fe_city_year,fe_fo,industry_name,financial_dep_china,financial_dep_us,hum_cap_us,phys_cap_us,liquidity_need_us,rd_intensity_us,foreign_share_us,constraint,constraint_1
0,2006,100037,长治市三宝生化药业有限公司,1404,Shanxi,2760,2710,27,0.0,5.0,1.210186,5.25578,5842.0,1.095,8599.0,140.0,19960.0,14869,31133,-5880,41957,67377,67377,17674,28954,46628,2179,0.38644,0.14403,0.62272,0.12763,0.01039,0.0,0.42973,0.03234,0.07526,0.0,0.0,-0.81536,500.0,500.0,NaN,0.0,500.0,500.0,0.0,NaN,NaN,NaN,NaN,0.4,NaN,NaN,400.0,40.0,NaN,529.0,3200.0,3750.0,17000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100000.0,NaN,1.0,4.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Shanxi,Changzhi,1404.0,0,0,0.0,NOT_SOE,NOT_FOREIGN,148,165,0,Medicines,-0.44,0.22,1.21,0.06,0.15,0.02,0.06,False,False


## Create lags

log(asset_tangibility_tot_asset) +
            log(lag_cashflow_to_tangible) +
            #log(lag_current_ratio) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)

In [15]:
df_final = (
    df
    .assign(
    **{
        f'lag_{c}': df.groupby(['firm'])[c].transform('shift') for c in
        ['cashflow_to_tangible',
                     'current_ratio',
         "quick_ratio",
                     'sales_tot_asset',
                     'liabilities_tot_asset',
                     "asset_tangibility_tot_asset",
                     "sales",
                     "total_asset",
                     "tfp_op","tfp_lp"]
    }
    )
    .dropna(subset = [
        'lag_cashflow_to_tangible',
                    #'lag_current_ratio',
                     'lag_sales_tot_asset',
                     'lag_liabilities_tot_asset',
                     #"lag_asset_tangibility_tot_asset",
                     #"lag_sales",
                     #"lag_total_asset",
                     #"lag_tfp_op",
                     #"lag_tfp_lp"
                     ])
    #.to_csv(df_path)
)

In [16]:
df_final.reindex(columns = [
    'lag_asset_tangibility_tot_asset',
    'lag_cashflow_to_tangible',
    'lag_current_ratio',
    "lag_quick_ratio",
    'lag_sales',
    'lag_total_asset',
    'lag_liabilities_tot_asset',
    'lag_sales_tot_asset',
    'lag_tfp_op',
    'lag_tfp_lp'
]).describe()

,lag_asset_tangibility_tot_asset,lag_cashflow_to_tangible,lag_current_ratio,lag_quick_ratio,lag_sales,lag_total_asset,lag_liabilities_tot_asset,lag_sales_tot_asset,lag_tfp_op,lag_tfp_lp
count,62992.000000,62992.000000,62992.000000,62992.000000,6.299200e+04,6.299200e+04,62992.000000,62992.000000,61312.000000,61312.000000
mean,0.498408,0.170537,1.157386,0.253412,3.083760e+05,4.165033e+05,0.578738,0.975131,0.809244,7.164143
std,0.195867,0.763335,4.428319,1.419629,1.711420e+06,2.303661e+06,0.196293,1.092976,0.343691,1.190627
min,0.000050,0.000010,0.000170,-0.680140,1.000000e+00,2.400000e+02,0.002130,0.000100,-7.108141,-1.214162
25%,0.353128,0.048500,0.516158,0.047720,2.060725e+04,2.865950e+04,0.450540,0.451967,0.660216,6.349331
50%,0.491860,0.092440,0.814610,0.131745,5.613600e+04,7.961000e+04,0.585870,0.715890,0.810496,7.102134
75%,0.641502,0.175592,1.210990,0.273053,1.667962e+05,2.421325e+05,0.713550,1.143603,0.963511,7.906987
max,0.999930,120.000000,659.200000,173.333330,1.170000e+08,1.647260e+08,9.685200,59.606030,10.258273,12.529512


In [17]:
df_final.shape

(62992, 113)

In [18]:
df_final[["SOE"]].isna().sum()

SOE    0
dtype: int64

In [19]:
(
    df_final
    .to_csv(df_path)
)

In [20]:
pd.DataFrame(schema)

,Name,Type
0,year,string
1,firm,string
2,name,string
3,geocode4_corr,string
4,province_en,string
...,...,...
77,hxxycsl,string
78,adcsl,string
79,eyhlcsl,string
80,dyhwcsl,string


## Schema Latex table

To rename a variable, please use the following template:

```
{
    'old':'XX',
    'new':'XX_1'
    }
```

if you need to pass a latex format with `\`, you need to duplicate it for instance, `\text` becomes `\\text:

```
{
    'old':'working\_capital\_i',
    'new':'\\text{working capital}_i'
    }
```

Then add it to the key `to_rename`

In [ ]:
add_to_dic = False
if add_to_dic:
    if os.path.exists("schema_table.json"):
        os.remove("schema_table.json")
    data = {'to_rename':[], 'to_remove':[]}
    dic_rename = [
        {
        'old':'working\_capital\_i',
        'new':'\\text{working capital}_i'
        },
        {
        'old':'periodTRUE',
        'new':'\\text{period}'
        },
        {
        'old':'tso2\_mandate\_c',
        'new':'\\text{policy mandate}_'
        },
    ]

    data['to_rename'].extend(dic_rename)
    with open('schema_table.json', 'w') as outfile:
        json.dump(data, outfile)

In [ ]:
sys.path.append(os.path.join(parent_path, 'utils'))
import latex.latex_beautify as lb
#%load_ext autoreload
#%autoreload 2

In [ ]:
options(warn=-1)
library(tidyverse)
library(lfe)
#library(lazyeval)
library('progress')
path = "../../../utils/latex/table_golatex.R"
source(path)

In [ ]:
%get df_path
df_final <- read_csv(df_path) %>%
mutate_if(is.character, as.factor) %>%
    mutate_at(vars(starts_with("fe")), as.factor)

In [ ]:
#df_final %>%select(so2)

## Table 1:XXX

$$
\begin{aligned}
\text{Write your equation}
\end{aligned}
$$


* Column 1: XXX
    * FE: 
        - fe 1: `XX`
        - fe 2: `XX`
        - fe 3: `XX`
* Column 2: XXX
    * FE: 
        - fe 1: `XX`
        - fe 2: `XX`
        - fe 3: `XX`
* Column 3: XXX
    * FE: 
        - fe 1: `XX`
        - fe 2: `XX`
        - fe 3: `XX`
* Column 4: XXX
    * FE: 
        - fe 1: `XX`
        - fe 2: `XX`
        - fe 3: `XX`

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.tex', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
(df_final
.assign(
share_o = lambda x: x['so2']/x['output']
)
.reindex(columns = [
    "share_o",
    'lag_cashflow_to_tangible',
                     'lag_current_ratio',
                     'lag_sales_tot_asset',
                     'lag_liabilities_tot_asset',
                     "lag_asset_tangibility_tot_asset",
                     "lag_sales",
                     "lag_total_asset",
                     "lag_tfp_op",
                     "lag_tfp_lp"
]).describe(percentiles=np.arange(0,1,.05))
)

### Replicate tables

- current_ratio
- cashflow
- quick_ratio
- return_to_sale
- coverage_ratio
- liquidity

In [ ]:
df_final.head(1)

In [ ]:
df_final.assign(eoutput = lambda x: x['output']/x['dso2_equip'])[[
    'so2','dso2_equip','eoutput'
]].corr()

In [ ]:
df_final['year'].unique()

In [ ]:
df_final.loc[lambda x: x['so2'] == 0].shape[0]/df_final.shape[0]

In [ ]:
df_final.loc[lambda x: x['cod'] == 0].shape[0]/df_final.shape[0]

In [ ]:
df_final.loc[lambda x: x['waste_water'] == 0].shape[0]/df_final.shape[0]

In [ ]:
df_final.filter(like='tfp')

Table 1

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
df_final.head(1)
# total_coal_used, waste_gas, nox, smoke_dust, soot

In [ ]:
%get path table
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_2 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
dep <- "Dependent variable: Pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2
),
    title="Determinant of pollution emissions",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

#multicolumn ={
#    'Eligible': 2,
#    'Non-Eligible': 1,
#    'All': 1,
#    'All benchmark': 1,
#}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& SO2', 'COD', "Waste water"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            #multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
t_0 <- felm(log(so2 +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <- felm(log(cod +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_2 <- felm(log(waste_water +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
dep <- "Dependent variable: Pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2
),
    title="Determinant of pollution emissions",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

#multicolumn ={
#    'Eligible': 2,
#    'Non-Eligible': 1,
#    'All': 1,
#    'All benchmark': 1,
#}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& SO2', 'COD', "Waste water"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            #multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
t_0 <- felm(log(so2 +1) ~ 
            rd_tot_asset_trick +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524)%>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)
t_1 <- felm(log(cod +1) ~ 
            rd_tot_asset_trick +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524)%>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)
t_2 <- felm(log(waste_water +1) ~ 
            rd_tot_asset_trick +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524)%>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)
dep <- "Dependent variable: Pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2
),
    title="Determinant of pollution emissions",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

#multicolumn ={
#    'Eligible': 2,
#    'Non-Eligible': 1,
#    'All': 1,
#    'All benchmark': 1,
#}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& SO2', 'COD', "Waste water"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            #multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
df_final

### Ownership

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(SOE == "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(SOE != "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(SOE == "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(SOE != "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(SOE == "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(SOE != "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and firm's ownership",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& SOE', 'No SOE', "SOE","No SOE", "SOE","No SOE"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(FOREIGN == "FOREIGN")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(FOREIGN != "FOREIGN")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(FOREIGN == "FOREIGN")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(FOREIGN != "FOREIGN")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(FOREIGN == "FOREIGN")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(FOREIGN != "FOREIGN")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and firm's ownership",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Foreign', 'No Foreign', "Foreign","No Foreign", "Foreign","No Foreign"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

## Firm's size

In [ ]:
df_final_size <- df_final %>%
  left_join(df_final %>%
              group_by(year, cic_adj) %>%
              summarise(output = quantile(output, 0.75),
                        sales = quantile(sales, 0.75),
                        employment = quantile(employment, 0.75),
                        total_asset = quantile(total_asset, 0.75)) %>%
              ungroup() %>%
              rename(
                s_output = output,
                s_sales = sales,
                s_employment = employment,
                s_total_asset = total_asset
              )
  ) %>%
  mutate(d_output = if_else(output > s_output, "Yes", "No"),
         d_sales = if_else(sales > s_sales, "Yes", "No"),
         d_employment = if_else(employment > s_employment, "Yes", "No"),
         d_total_asset = if_else(total_asset > s_total_asset, "Yes", "No"))

In [ ]:
dim(df_final_size %>% filter(d_output == "Yes"))

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_output == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_output != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_output == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_output != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_output == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_output != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and firm's size",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Large', 'No Large', "Large","No Large", "Large","No Large"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and firm's size",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Large', 'No Large', "Large","No Large", "Large","No Large"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_sales == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_sales != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_sales == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_sales != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_sales == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_sales != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and firm's size",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Large', 'No Large', "Large","No Large", "Large","No Large"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and firm's size",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Large', 'No Large', "Large","No Large", "Large","No Large"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_total_asset == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_total_asset != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_total_asset == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_total_asset != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_total_asset == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_total_asset != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and firm's size",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Large', 'No Large', "Large","No Large", "Large","No Large"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

### TZC/SPZ

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(tcz == 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(tcz != 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(tcz == 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(tcz !=1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(tcz == 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(tcz != 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and city type",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& TCZ', 'No TCZ', "TCZ","No TCZ", "TCZ","No TCZ"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(spz == 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(spz != 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(spz == 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(spz !=1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(spz == 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(spz != 1)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and city type",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& SPZ', 'No SPZ', "SPZ","No SPZ", "SPZ","No SPZ"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

### Constraint

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
df_final.head(1)

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint == TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint != TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint == TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint !=TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint == TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(asset_tangibility_tot_asset) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint != TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and Financial constraint",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Contraint', 'No Contraint', "Contraint","No Contraint", "Contraint","No Contraint"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
%get path table
#### SO2
t_0 <- felm(log(so2 +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint == TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1 <-felm(log(so2 +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint != TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

### COD
t_2 <- felm(log(cod +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint == TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_3 <-felm(log(cod +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint !=TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
### Waste water
t_4 <- felm(log(waste_water +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint == TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_5 <-felm(log(waste_water +1) ~ 
            log(tfp_op) +
            log(sales) +
            log(total_asset) +
            log(lag_liabilities_tot_asset) +
            log(lag_sales_tot_asset)
            | firm+year+geocode4_corr|0 | firm, df_final%>%filter(constraint != TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Pollution emissions and Financial constraint",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Contraint', 'No Contraint', "Contraint","No Contraint", "Contraint","No Contraint"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

### Transmission channel

Faire avec taille

Regarder l'hétérogénéity, SOE/foreign/taille

- Changer de polluants

Est ce que l'accès a la finance interne affecte les emissions. 

Une entreprise publique/large est moins sensible a la contrainte de crédit.

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
t_0 <-felm(log(asset_tangibility_tot_asset) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1<-felm(rd_tot_asset_trick ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524) %>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)
t_2<-felm(log(tfp_op) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524) ,
            exactDOF = TRUE)
dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2
),
    title="Transmission channel",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
)

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Asset tangilbility', 'R\&D', "TFP"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            #multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
#- current_ratio
#- cashflow
#- quick_ratio
#- return_to_sale
#- coverage_ratio
#- liquidity

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
df_final['liquidity'].describe()

In [ ]:
t_0 <-felm(log(asset_tangibility_tot_asset) ~ 
            log(current_ratio) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)
t_1<-felm(rd_tot_asset_trick ~ 
            log(current_ratio) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524) %>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)
t_2<-felm(log(tfp_op) ~ 
            log(current_ratio) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524) ,
            exactDOF = TRUE)
dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2
),
    title="Transmission channel",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
)

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'SO2': 2,
    'COD': 2,
    'Waste water': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Asset tangilbility', 'R\&D', "TFP"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            #multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
t_0 <-felm(log(asset_tangibility_tot_asset) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% filter(SOE == "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

t_1 <-felm(log(asset_tangibility_tot_asset) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% filter(SOE != "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

t_2<-felm(rd_tot_asset_trick ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524) %>% filter(SOE == "SOE")%>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)

t_3<-felm(rd_tot_asset_trick ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524) %>% filter(SOE != "SOE")%>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)

t_4<-felm(log(tfp_op) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% filter(SOE == "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524) ,
            exactDOF = TRUE)

t_5<-felm(log(tfp_op) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% filter(SOE != "SOE")%>% 
            filter(lag_cashflow_to_tangible <0.837524) ,
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Transmission channel",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
)

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'Asset tangilbility': 2,
    'R\&D': 2,
    'TFP': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& SOE', 'No SOE', "SOE","No SOE", "SOE","No SOE"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
t_0 <-felm(log(asset_tangibility_tot_asset) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% filter(constraint == TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

t_1 <-felm(log(asset_tangibility_tot_asset) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% filter(constraint != TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

t_2<-felm(rd_tot_asset_trick ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524) %>% filter(constraint == TRUE)%>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)

t_3<-felm(rd_tot_asset_trick ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% 
            filter(lag_cashflow_to_tangible <0.837524) %>% filter(constraint != TRUE)%>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)

t_4<-felm(log(tfp_op) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% filter(constraint == TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524) ,
            exactDOF = TRUE)

t_5<-felm(log(tfp_op) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final%>% filter(constraint != TRUE)%>% 
            filter(lag_cashflow_to_tangible <0.837524) ,
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Transmission channel",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
)

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'Asset tangilbility': 2,
    'R\&D': 2,
    'TFP': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Contraint', 'No Contraint', "Contraint","No Contraint", "Contraint","No Contraint"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

In [ ]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [ ]:
t_0 <-felm(log(asset_tangibility_tot_asset) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

t_1 <-felm(log(asset_tangibility_tot_asset) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE)

t_2<-felm(rd_tot_asset_trick ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% 
            filter(lag_cashflow_to_tangible <0.837524) %>% filter(d_employment == "Yes")%>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)

t_3<-felm(rd_tot_asset_trick ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% 
            filter(lag_cashflow_to_tangible <0.837524) %>% filter(d_employment != "Yes")%>% filter(year %in% list("2005","2006", "2007")),
            exactDOF = TRUE)

t_4<-felm(log(tfp_op) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment == "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524) ,
            exactDOF = TRUE)

t_5<-felm(log(tfp_op) ~ 
            log(cashflow_to_tangible) + 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% filter(d_employment != "Yes")%>% 
            filter(lag_cashflow_to_tangible <0.837524) ,
            exactDOF = TRUE)

dep <- "Dependent variable: pollution emissions"
fe1 <- list(
    c("firm", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("year", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"),
    c("city", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3, t_4, t_5
),
    title="Transmission channel",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
)

In [ ]:
summary(
felm(log(asset_tangibility_tot_asset) ~ 
            log(cashflow_to_tangible) * log(total_asset)+ 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE))

In [ ]:
summary(
felm(log(asset_tangibility_tot_asset) ~ 
            log(cashflow_to_tangible) * log(employment)+ 
            log(liabilities_tot_asset) + 
            log(total_asset) + 
            log(age)
            | firm+year+geocode4_corr|0 | firm, df_final_size%>% 
            filter(lag_cashflow_to_tangible <0.837524),
            exactDOF = TRUE))

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

multicolumn ={
    'Asset tangilbility': 2,
    'R\&D': 2,
    'TFP': 2
}

#multi_lines_dep = '(city/product/trade regime/year)'
new_r = ['& Large', 'No Large', "Large","No Large", "Large","No Large"]
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            new_row= new_r,
            multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

# Replicate one internal finance


-> prendre que asset tangilbilty et mettre les polluants, on explique que asset tangiblity

Mecanisme:

Cashflow

- Journal:
    - Environment and development economics: https://www.cambridge.org/core/journals/environment-and-development-economics
    - Review of environmental econ and policy: https://www.journals.uchicago.edu/toc/reep/current

In [ ]:
%get path table
t_0 <- felm(YYY ~XXX
            | FE|0 | CLUSTER, df_final %>% filter(XXX == 'YYY'),
            exactDOF = TRUE)

t_0 <- felm(YYY ~XXX
            | FE|0 | CLUSTER, df_final %>% filter(XXX != 'YYY'),
            exactDOF = TRUE)

t_2 <- felm(kYYY ~XXX
            | FE|0 | CLUSTER, df_final,
            exactDOF = TRUE)

t_3 <- felm(kYYY ~XXX
            | FE|0 | CLUSTER, df_final,
            exactDOF = TRUE)
            
dep <- "Dependent variable: YYYY"
fe1 <- list(
    c("XXXXX", "Yes", "Yes", "No", "No"),
    
    c("XXXXX", "Yes", "Yes", "No", "No"),
    
    c("XXXXX","Yes", "Yes", "Yes", "No"),
    
    c("XXXXX","No", "No", "Yes", "Yes"),
    
    c("XXXXX","No", "No", "Yes", "Yes"),
    
    c("XXXXX", "No", "No", "No", "Yes")
             )

table_1 <- go_latex(list(
    t_0,t_1, t_2, t_3
),
    title="TITLE",
    dep_var = dep,
    addFE=fe1,
    save=TRUE,
    note = FALSE,
    name=path
) 

In [ ]:
tbe1  = "This table estimates eq(3). " \
"Heteroskedasticity-robust standard errors" \
"clustered at the product level appear inparentheses."\
"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

#multicolumn ={
#    'Eligible': 2,
#    'Non-Eligible': 1,
#    'All': 1,
#    'All benchmark': 1,
#}

#multi_lines_dep = '(city/product/trade regime/year)'
#new_r = ['& test1', 'test2']
lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
            #new_row= new_r,
            #multicolumn = multicolumn,
            table_nte = tbe1,
            jupyter_preview = True,
            resolution = 150,
            folder = folder)

# Generate reports

In [ ]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp
import sys
path = os.getcwd()
parent_path = str(Path(path).parent.parent.parent)
sys.path.append(os.path.join(parent_path, 'utils'))
import make_toc
import create_report

In [ ]:
name_json = 'parameters_ETL_pollution_credit_constraint.json'
path_json = os.path.join(str(Path(path).parent.parent), 'utils',name_json)

In [ ]:
create_report.create_report(extension = "html", keep_code = False, notebookname = None)

In [ ]:
### Update TOC in Github
for p in [parent_path,
          str(Path(path).parent),
          #os.path.join(str(Path(path).parent), "00_download_data_from"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis", "00_statistical_exploration"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis", "01_model_estimation"),
         ]:
    try:
        os.remove(os.path.join(p, 'README.md'))
    except:
        pass
    path_parameter = os.path.join(parent_path,'utils', name_json)
    md_lines =  make_toc.create_index(cwd = p, path_parameter = path_parameter)
    md_out_fn = os.path.join(p,'README.md')
    
    if p == parent_path:
    
        make_toc.replace_index(md_out_fn, md_lines, Header = os.path.basename(p).replace('_', ' '), add_description = True, path_parameter = path_parameter)
    else:
        make_toc.replace_index(md_out_fn, md_lines, Header = os.path.basename(p).replace('_', ' '), add_description = False)

In [ ]:
!jupyter nbconvert --no-input --to html 08_firm_level_estimation_pollution.ipynb